## 注意事项
* 将文本符号化成句子之后就应该padding一下句子
* 先清理符号还是先进行讲文本分成句子
* 1.获取文本
* 2.将文本分成句子
* 3.将句子padding一下
* 4.将句子分成词
* 5.将词映射到vector里

In [2]:
import nltk
import numpy as np
import re
import pickle
import itertools
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.preprocessing.sequence import pad_sequences
import re
from os import listdir
import string
from nltk.corpus import stopwords

In [3]:
#loac doc into memory
def load_doc(filepath):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    #open the file as read only
    file = open(filepath,'r')
    # read all text
    x_text = file.read()
    #close the file
    file.close()
    return x_text

def load_data_and_labels(filepath):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(filepath,'r', encoding='UTF-8').readlines())
    positive_examples = [s.strip() for s in positive_examples]
    # Split by words
    x_text = [s.split(" ") for s in positive_examples]
    # Generate labels
    return x_text

def clean_doc(doc):
    """Tokenization/string cleaning for all datasets """
    #split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower

def sent_tokenize(doc):
    sent_text = nltk.sent_tokenize(doc) # this gives you a list of sentences
    return sent_text
    
def word_tokenize(sent):
    tokenized_text = nltk.word_tokenize(sent)  # this gives you a list of words
    tokenized_text = [token.lower() for token in tokenized_text]  # optional: convert all words to lower case
    return tokenized_text  

def pad_sentences(sentences, sequence_length=None):
    """
    Pads all sentences to the same length. The length is defined by the longest sentence.
    Returns padded sentences.
    """
    padding_word = "<PAD/>"

    length = [len(x) for x in sentences]
    if sequence_length == None:
        sequence_length = int(np.percentile(length, 90))

    print('max sentence length is %d' % sequence_length)
    padded_sentences = []
    sentence_length = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        if num_padding > 0:
            new_sentence = sentence + [padding_word] * num_padding
            padded_sentences.append(new_sentence)
            sentence_length.append(len(sentence))
        else:
            padded_sentences.append(sentence[:sequence_length])
    return [sequence_length, padded_sentences, sentence_length]


In [16]:
x_text1 = load_data_and_labels('D:/code_stock/SA_Code/data/aclImdb/train/neg/0_3.txt')
type(x_text1)

list

In [15]:
x_text2 = load_doc('D:/code_stock/SA_Code/data/aclImdb/train/neg/0_3.txt')
x_text2 = clean_doc(x_text2)
type(x_text2)

list

In [ ]:
#specify the directory to load
directory = 'D:/code_stock/SA_Code/data/aclImdb/train/neg'
#walk through all files in the folder
for filepath in listdir(directory):
    #skip files that do not have the right extension
    if not filepath.endswith(".txt"):
        continue
    #create the full filepath to open
    path = directory + '/' + filepath
    #load and label the document
    doc = load_doc(path)
    print('Loading %s'%filepath)

In [ ]:
for filepath in listdir(directory):
    #skip files that do not have the right extension
    if not filepath.endswith(".txt"):
        continue
    #create the full filepath to open
    path = directory + '/' + filepath
    #load and label the document
    doc = clean_doc(path)
    print('Tokenizing %s'%filepath)

In [ ]:
x_test1 = load_doc('D:/code_stock/SA_Code/data/aclImdb/train/neg/0_3.txt')
type(x_test1)


In [ ]:
x_test2 = load_data_and_labels('D:/code_stock/SA_Code/data/aclImdb/train/neg/0_3.txt')
type(x_test2)
x_test2